In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.metrics import AUC
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, Activation, BatchNormalization, LayerNormalization, GaussianDropout, SpatialDropout1D, AveragePooling1D 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import f1_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split



In [44]:
df_1 = pd.read_csv("C:/03 Master/FS22/MLHC/Project/archive/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("C:/03 Master/FS22/MLHC/Project/archive/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [45]:
aucroc = AUC(curve='ROC')
aucpr = AUC(curve='PR')

In [46]:
def get_model1():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=4)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=4)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[aucroc,aucpr,'acc'])
    model.summary()
    return model

In [47]:
def get_model2():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = AveragePooling1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = AveragePooling1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = AveragePooling1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[aucroc,aucpr,'acc'])
    model.summary()
    return model

In [48]:
def get_model3():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = SpatialDropout1D(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = SpatialDropout1D(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = SpatialDropout1D(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    
    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[aucroc,aucpr,'acc'])
    model.summary()
    return model

In [49]:
def get_model4():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    
    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[aucroc,aucpr,'acc'])
    model.summary()
    return model

In [50]:
def get_model5():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = GaussianDropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = GaussianDropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = GaussianDropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = GaussianDropout(rate=0.2)(img_1)
    
    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[aucroc,aucpr,'acc'])
    model.summary()
    return model

In [51]:
def get_model6():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [52]:
def get_model7():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.tanh, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.tanh, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.tanh, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.tanh, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.tanh, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.tanh, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.tanh, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.tanh, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=[aucroc,aucpr,'acc'])
    model.summary()
    return model

In [53]:
def get_model8():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, padding="valid")(img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, padding="valid")(img_1)
    img_1 = BatchNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [54]:
def get_model9():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, padding="valid")(img_1)
    img_1 = LayerNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = LayerNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, padding="valid")(img_1)
    img_1 = LayerNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, padding="valid")(img_1)
    img_1 = LayerNormalization()(img_1)
    img_1 = Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

# Model 1

In [55]:
model1 = get_model1()
file_path = "model1_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_97 (Conv1D)           (None, 183, 16)           96        
_________________________________________________________________
conv1d_98 (Conv1D)           (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 44, 16)            0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 44, 16)            0         
_________________________________________________________________
conv1d_99 (Conv1D)           (None, 42, 32)            1568      
_________________________________________________________________
conv1d_100 (Conv1D)          (None, 40, 32)            310

In [57]:
history1 = model1.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model1.load_weights(file_path)

pred_test = model1.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)


Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc did not improve from 0.99399
10476/10476 - 6s - loss: 0.0221 - auc_6: 0.9996 - auc_7: 0.9999 - acc: 0.9919 - val_loss: 0.0219 - val_auc_6: 0.9995 - val_auc_7: 0.9998 - val_acc: 0.9940
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.99399
10476/10476 - 6s - loss: 0.0193 - auc_6: 0.9997 - auc_7: 0.9999 - acc: 0.9925 - val_loss: 0.0218 - val_auc_6: 0.9995 - val_auc_7: 0.9998 - val_acc: 0.9940
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.99399
10476/10476 - 6s - loss: 0.0200 - auc_6: 0.9994 - auc_7: 0.9997 - acc: 0.9940 - val_loss: 0.0218 - val_auc_6: 0.9995 - val_auc_7: 0.9998 - val_acc: 0.9940
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.99399

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
10476/10476 - 6s - loss: 0.0205 - auc_6: 0.9994 - auc_7: 0.9997 - acc: 0.9927 - val_loss: 0.0218 - val_auc_6: 0.9996 - val_auc_7: 0.9998 - val_acc: 

In [58]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9959648706384999 
Test accuracy score : 0.9941600824458949 


# Model 2

In [59]:
model2 = get_model2()
file_path = "model2_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_103 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_104 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
average_pooling1d_3 (Average (None, 89, 16)            0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 89, 16)            0         
_________________________________________________________________
conv1d_105 (Conv1D)          (None, 87, 32)            1568      
_________________________________________________________________
conv1d_106 (Conv1D)          (None, 85, 32)            310

In [60]:
history2 = model2.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model2.load_weights(file_path)

pred_test = model2.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.81717, saving model to model2_cnn_ptbdb2.h5
10476/10476 - 18s - loss: 0.5055 - auc_6: 0.7461 - auc_7: 0.8870 - acc: 0.7509 - val_loss: 0.4021 - val_auc_6: 0.8710 - val_auc_7: 0.9391 - val_acc: 0.8172
Epoch 2/1000

Epoch 00002: val_acc improved from 0.81717 to 0.86867, saving model to model2_cnn_ptbdb2.h5
10476/10476 - 10s - loss: 0.3638 - auc_6: 0.8899 - auc_7: 0.9510 - acc: 0.8396 - val_loss: 0.3090 - val_auc_6: 0.9281 - val_auc_7: 0.9644 - val_acc: 0.8687
Epoch 3/1000

Epoch 00003: val_acc improved from 0.86867 to 0.89099, saving model to model2_cnn_ptbdb2.h5
10476/10476 - 10s - loss: 0.3037 - auc_6: 0.9241 - auc_7: 0.9667 - acc: 0.8729 - val_loss: 0.2624 - val_auc_6: 0.9510 - val_auc_7: 0.9772 - val_acc: 0.8910
Epoch 4/1000

Epoch 00004: val_acc improved from 0.89099 to 0.92103, saving model to model2_cnn_ptbdb2.h5
10476/10476 - 10s - loss: 0.2482 - auc_6: 0.9492 - auc_7: 0.97

In [61]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9893390191897655 
Test accuracy score : 0.9845413947097218 


# Model 3

In [62]:
model3 = get_model3()
file_path = "model3_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_111 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_112 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 89, 16)            0         
_________________________________________________________________
conv1d_113 (Conv1D)          (None, 87, 32)            1568      
_________________________________________________________________
conv1d_114 (Conv1D)          (None, 85, 32)            310

In [64]:
history3 = model3.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model3.load_weights(file_path)

pred_test = model3.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc did not improve from 0.98798
10476/10476 - 11s - loss: 0.0152 - auc_6: 0.9992 - auc_7: 0.9995 - acc: 0.9952 - val_loss: 0.0431 - val_auc_6: 0.9949 - val_auc_7: 0.9958 - val_acc: 0.9880
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.98798
10476/10476 - 11s - loss: 0.0141 - auc_6: 0.9993 - auc_7: 0.9995 - acc: 0.9958 - val_loss: 0.0429 - val_auc_6: 0.9950 - val_auc_7: 0.9958 - val_acc: 0.9880
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.98798
10476/10476 - 11s - loss: 0.0163 - auc_6: 0.9994 - auc_7: 0.9996 - acc: 0.9947 - val_loss: 0.0428 - val_auc_6: 0.9949 - val_auc_7: 0.9958 - val_acc: 0.9871
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.98798

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
10476/10476 - 11s - loss: 0.0148 - auc_6: 0.9993 - auc_7: 0.9995 - acc: 0.9946 - val_loss: 0.0427 - val_auc_6: 0.9950 - val_auc_7: 0.9958 - val_a

In [65]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9928639391056137 
Test accuracy score : 0.9896942631398145 


# Model 4

In [67]:
model4 = get_model4()
file_path = "model4_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_119 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_120 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
conv1d_121 (Conv1D)          (None, 87, 32)            1568      
_________________________________________________________________
conv1d_122 (Conv1D)          (None, 85, 32)            3104      
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 42, 32)            0  

In [68]:
history4 = model4.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model4.load_weights(file_path)

pred_test = model4.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.83605, saving model to model4_cnn_ptbdb2.h5
10476/10476 - 19s - loss: 0.4839 - auc_6: 0.7748 - auc_7: 0.9025 - acc: 0.7617 - val_loss: 0.4022 - val_auc_6: 0.8979 - val_auc_7: 0.9487 - val_acc: 0.8361
Epoch 2/1000

Epoch 00002: val_acc improved from 0.83605 to 0.87811, saving model to model4_cnn_ptbdb2.h5
10476/10476 - 10s - loss: 0.2593 - auc_6: 0.9461 - auc_7: 0.9770 - acc: 0.8954 - val_loss: 0.2776 - val_auc_6: 0.9560 - val_auc_7: 0.9805 - val_acc: 0.8781
Epoch 3/1000

Epoch 00003: val_acc improved from 0.87811 to 0.93820, saving model to model4_cnn_ptbdb2.h5
10476/10476 - 10s - loss: 0.1680 - auc_6: 0.9772 - auc_7: 0.9903 - acc: 0.9364 - val_loss: 0.1470 - val_auc_6: 0.9849 - val_auc_7: 0.9933 - val_acc: 0.9382
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.93820
10476/10476 - 10s - loss: 0.1303 - auc_6: 0.9857 - auc_7: 0.9937 - acc: 0.9497 - val_loss: 0.2585 - val_

In [69]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.993100166547704 
Test accuracy score : 0.9900377877018207 


# Model 5

In [70]:
model5 = get_model5()
file_path = "model5_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_127 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_128 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_37 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
gaussian_dropout_4 (Gaussian (None, 89, 16)            0         
_________________________________________________________________
conv1d_129 (Conv1D)          (None, 87, 32)            1568      
_________________________________________________________________
conv1d_130 (Conv1D)          (None, 85, 32)            310

In [71]:
history5 = model5.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model5.load_weights(file_path)

pred_test = model5.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.77940, saving model to model5_cnn_ptbdb2.h5
10476/10476 - 21s - loss: 0.5140 - auc_6: 0.7292 - auc_7: 0.8836 - acc: 0.7299 - val_loss: 0.4536 - val_auc_6: 0.8660 - val_auc_7: 0.9349 - val_acc: 0.7794
Epoch 2/1000

Epoch 00002: val_acc improved from 0.77940 to 0.87811, saving model to model5_cnn_ptbdb2.h5
10476/10476 - 11s - loss: 0.3835 - auc_6: 0.8731 - auc_7: 0.9441 - acc: 0.8326 - val_loss: 0.3190 - val_auc_6: 0.9159 - val_auc_7: 0.9575 - val_acc: 0.8781
Epoch 3/1000

Epoch 00003: val_acc improved from 0.87811 to 0.90558, saving model to model5_cnn_ptbdb2.h5
10476/10476 - 10s - loss: 0.3097 - auc_6: 0.9153 - auc_7: 0.9608 - acc: 0.8799 - val_loss: 0.2614 - val_auc_6: 0.9411 - val_auc_7: 0.9698 - val_acc: 0.9056
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.90558
10476/10476 - 10s - loss: 0.2693 - auc_6: 0.9369 - auc_7: 0.9706 - acc: 0.8955 - val_loss: 0.2477 - val_

In [72]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9919239904988124 
Test accuracy score : 0.9883201648917898 


# Model 6

In [73]:
model6 = get_model6()
file_path = "model6_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_135 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_136 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
dropout_36 (Dropout)         (None, 179, 16)           0         
_________________________________________________________________
conv1d_137 (Conv1D)          (None, 177, 32)           1568      
_________________________________________________________________
conv1d_138 (Conv1D)          (None, 175, 32)           3104      
_________________________________________________________________
dropout_37 (Dropout)         (None, 175, 32)           0  

In [74]:
history6 = model6.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model6.load_weights(file_path)

pred_test = model6.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.69957, saving model to model6_cnn_ptbdb2.h5
10476/10476 - 55s - loss: 0.5530 - acc: 0.7237 - val_loss: 0.5477 - val_acc: 0.6996
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.69957
10476/10476 - 44s - loss: 0.5003 - acc: 0.7244 - val_loss: 0.4535 - val_acc: 0.6996
Epoch 3/1000

Epoch 00003: val_acc improved from 0.69957 to 0.70815, saving model to model6_cnn_ptbdb2.h5
10476/10476 - 44s - loss: 0.4578 - acc: 0.7437 - val_loss: 0.4881 - val_acc: 0.7082
Epoch 4/1000

Epoch 00004: val_acc improved from 0.70815 to 0.81888, saving model to model6_cnn_ptbdb2.h5
10476/10476 - 44s - loss: 0.4268 - acc: 0.7799 - val_loss: 0.3887 - val_acc: 0.8189
Epoch 5/1000

Epoch 00005: val_acc improved from 0.81888 to 0.83433, saving model to model6_cnn_ptbdb2.h5
10476/10476 - 44s - loss: 0.3913 - acc: 0.8089 - val_loss: 0.3634 - val_acc: 0.8343
Epoch 6/1000

Epoch 00006: val_acc improved fr

In [75]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9691358024691359 
Test accuracy score : 0.9553418069391961 


# Model 7

In [76]:
model7 = get_model7()
file_path = "model7_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_143 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_144 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 89, 16)            0         
_________________________________________________________________
conv1d_145 (Conv1D)          (None, 87, 32)            1568      
_________________________________________________________________
conv1d_146 (Conv1D)          (None, 85, 32)            310

In [77]:
history7 = model7.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model7.load_weights(file_path)

pred_test = model7.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.87983, saving model to model7_cnn_ptbdb2.h5
10476/10476 - 27s - loss: 0.4364 - auc_6: 0.8297 - auc_7: 0.9255 - acc: 0.7920 - val_loss: 0.3125 - val_auc_6: 0.9318 - val_auc_7: 0.9699 - val_acc: 0.8798
Epoch 2/1000

Epoch 00002: val_acc improved from 0.87983 to 0.90815, saving model to model7_cnn_ptbdb2.h5
10476/10476 - 13s - loss: 0.2662 - auc_6: 0.9445 - auc_7: 0.9780 - acc: 0.8835 - val_loss: 0.1997 - val_auc_6: 0.9732 - val_auc_7: 0.9886 - val_acc: 0.9082
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.90815
10476/10476 - 13s - loss: 0.1999 - auc_6: 0.9687 - auc_7: 0.9874 - acc: 0.9186 - val_loss: 0.2611 - val_auc_6: 0.9857 - val_auc_7: 0.9938 - val_acc: 0.8824
Epoch 4/1000

Epoch 00004: val_acc improved from 0.90815 to 0.93991, saving model to model7_cnn_ptbdb2.h5
10476/10476 - 13s - loss: 0.1694 - auc_6: 0.9776 - auc_7: 0.9910 - acc: 0.9318 - val_loss: 0.1379 - val_

In [78]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9940546967895363 
Test accuracy score : 0.9914118859498454 


# Model 8

In [79]:
model8 = get_model8()
file_path = "model8_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_151 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_152 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
batch_normalization_5 (Batch (None, 179, 16)           64        
_________________________________________________________________
activation_9 (Activation)    (None, 179, 16)           0         
_________________________________________________________________
max_pooling1d_43 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 89, 16)            0  

In [80]:
history8 = model8.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model8.load_weights(file_path)

pred_test = model8.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.30043, saving model to model8_cnn_ptbdb2.h5
10476/10476 - 162s - loss: 0.4431 - acc: 0.7868 - val_loss: 1.3708 - val_acc: 0.3004
Epoch 2/1000

Epoch 00002: val_acc improved from 0.30043 to 0.70043, saving model to model8_cnn_ptbdb2.h5
10476/10476 - 32s - loss: 0.2833 - acc: 0.8786 - val_loss: 0.9399 - val_acc: 0.7004
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.70043
10476/10476 - 31s - loss: 0.2180 - acc: 0.9110 - val_loss: 1.1721 - val_acc: 0.6206
Epoch 4/1000

Epoch 00004: val_acc improved from 0.70043 to 0.94163, saving model to model8_cnn_ptbdb2.h5
10476/10476 - 33s - loss: 0.1780 - acc: 0.9311 - val_loss: 0.1668 - val_acc: 0.9416
Epoch 5/1000

Epoch 00005: val_acc did not improve from 0.94163
10476/10476 - 33s - loss: 0.1581 - acc: 0.9388 - val_loss: 0.1630 - val_acc: 0.9330
Epoch 6/1000

Epoch 00006: val_acc did not improve from 0.94163
10476/10476 - 35s - los

In [81]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9900709219858156 
Test accuracy score : 0.9855719683957403 


# Model 9

In [82]:
model9 = get_model9()
file_path = "model9_cnn_ptbdb2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_159 (Conv1D)          (None, 183, 16)           96        
_________________________________________________________________
conv1d_160 (Conv1D)          (None, 179, 16)           1296      
_________________________________________________________________
layer_normalization_4 (Layer (None, 179, 16)           32        
_________________________________________________________________
activation_13 (Activation)   (None, 179, 16)           0         
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 89, 16)            0  

In [83]:
history9 = model9.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model9.load_weights(file_path)

pred_test = model9.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.69957, saving model to model9_cnn_ptbdb2.h5
10476/10476 - 152s - loss: 0.5469 - acc: 0.7213 - val_loss: 0.5593 - val_acc: 0.6996
Epoch 2/1000

Epoch 00002: val_acc did not improve from 0.69957
10476/10476 - 40s - loss: 0.5307 - acc: 0.7227 - val_loss: 0.5541 - val_acc: 0.6996
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.69957
10476/10476 - 41s - loss: 0.5270 - acc: 0.7212 - val_loss: 0.5522 - val_acc: 0.6996
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.69957

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
10476/10476 - 65s - loss: 0.5259 - acc: 0.7241 - val_loss: 0.5525 - val_acc: 0.6910
Epoch 5/1000

Epoch 00005: val_acc did not improve from 0.69957
10476/10476 - 45s - loss: 0.5207 - acc: 0.7245 - val_loss: 0.5501 - val_acc: 0.6996
Epoch 6/1000

Epoch 00006: val_acc did not improve from 0.69957
10476/10476 - 39s - loss:

In [84]:
f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.8386195890684222 
Test accuracy score : 0.7220886293369976 
